# Model Fine Tuning
----
**Objective**: In this notebook, you will get to fine tune a transformer model on MRPC dataset. You will get to go through the whole process of transfer learning from loading the dataset, loading the model, compiling the model, training the model, and finally evaluating your results.

NOTE: Make sure to change the runtime from CPU to TPU or GPU for faster training

## Install Libraries
Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


## Import Dataset
___
In this section we will use as an example the MRPC (Microsoft Research Paraphrase Corpus) dataset. The dataset consists of 5,801 pairs of sentences, with a label indicating if they are paraphrases or not (i.e., if both sentences mean the same thing).
This is one of the 10 datasets composing the GLUE benchmark, which is an academic benchmark that is used to measure the performance of ML models across 10 different text classification tasks.


**Question 1**: Use the "load_datasets" function to load the "mrpc" dataset from the "glue" benchmark.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

# Load the MRPC dataset from the GLUE benchmark
raw_datasets = load_dataset("glue", "mrpc")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Define the data collator for padding sequences
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

## Pre-processing data

First, we need to tokenize our data. We will do so using the BERT tokenizer. We will use the AutoTokenizer with truncation.
This will truncate token by token, removing a token from the longest sequence in the pair if a pair of sequences (or a batch of pairs) is provided.

**Question 2:** Use the .map() function to apply the tokenization functionon the raw dataset. Hint: set the batched flag to True for faster processing.

In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


# Apply tokenization function using .map() function
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    num_proc=4
)

Map (num_proc=4):   0%|          | 0/3668 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/408 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1725 [00:00<?, ? examples/s]

Then, we need to pad our data so that all input sequences have the same lengths.

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

Lets take a training example to see what our dataset looks like.

**Question 3:** Display an example from the training section of the dataset. Hint: The tokenized_datasets variable is a nested dictionary where the main keys are "train", "test", and "validate"

In [ ]:
# Access the tokenized training dataset
training_dataset = tokenized_datasets["train"]

# Display an example from the training dataset
example_index = 0
example = training_dataset[example_index]

print("Example from the training dataset:")
print("Input IDs:", example["input_ids"])
print("Attention Mask:", example["attention_mask"])
print("Token Type IDs:", example["token_type_ids"])

Example from the training dataset:
Input IDs: [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102]
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Token Type IDs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


Finally, we will prepare our training and validation datasets by transforming them into tensorflow datasets.

In [ ]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:401: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


## Model Preperation
---
In this section, we will load the model and equip it with a classifier head. This model is already pre-trained and we will fine tune it with the dataset for paraphrasing classification that we prepared.

### Load Classifier Head

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Compile the model

We will compile the model using SparseCategoricalCrossentropyloss, which is when there are two or more label classes that are not one hot encded.

**Question 4:** Compile the model using the adam optimizer, SparseCategoricalCrossentropy loss (with from_logits flag set to True) and displaying the accuracy metric.

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

# Load the pre-trained model with a sequence classification head
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=3e-5),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=[SparseCategoricalAccuracy('accuracy')]
)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Train the model

Now we will train our model using the processing training and validation sets.

**Question 5:** Fit the model on the training and validation sections that we prepared earlier.

In [ ]:
# Fit the model on the training and validation datasets
history = model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=3,  # Number of epochs
    verbose=1,  # Verbosity mode
    callbacks=None
)

Epoch 1/3
459/459 [==============================] - 133s 182ms/step - loss: 0.5173 - accuracy: 0.7445 - val_loss: 0.3827 - val_accuracy: 0.8382
Epoch 2/3
459/459 [==============================] - 73s 158ms/step - loss: 0.2899 - accuracy: 0.8874 - val_loss: 0.3382 - val_accuracy: 0.8603
Epoch 3/3
459/459 [==============================] - 73s 158ms/step - loss: 0.1146 - accuracy: 0.9597 - val_loss: 0.4964 - val_accuracy: 0.8456


## Evaluating the model

Let's test how accurate the models are! In this section, we will predict on the testing dataset to check the model's performance when it comes to classifying paraphrased sentences on unseen data.

In [ ]:
preds = model.predict(tf_test_dataset)["logits"]

216/216 [==============================] - 15s 53ms/step


In [ ]:
import numpy as np
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

(1725, 2) (1725,)


In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=class_preds, references=raw_datasets["test"]["label"])

{'accuracy': 0.8365217391304348, 'f1': 0.88}